In [1]:
import heapq
import time, random
import sys

In [2]:
class State:
    def __init__(self, initial_state):
        self.current_state = initial_state
        
    def __lt__(self, other) -> bool:
        pass

    def __eq__(self, other) -> bool:
        pass
    
    def __str__(self):
        return ' '.join(map(str, self.current_state))
    
    def count_conflicts(self, current_state):
        count = 0
        n = len(current_state)
        for i in range(n):
            for j in range(i + 1, n):
                if current_state[i] == current_state[j] or abs(current_state[i] - current_state[j]) == abs(j - i):
                    count += 1
        return count

    def get_successors(self, current_state):
        successors = []
        n = len(current_state)
        for i in range(n):
            for j in range(n):
                if j != current_state[i]:
                    new_state = current_state.copy()
                    new_state[i] = j
                    successors.append(new_state)
        return successors
    
    def goal_test(self, current_state):
        visited = set()
        n = len(current_state)
        for i in range(n):
            if current_state[i] in visited:
                return False
            visited.add(current_state[i])
            for j in range(i + 1, n):
                if abs(current_state[i] - current_state[j]) == abs(j - i):
                    return False
        return True
    
class AStarState(State):
    def __init__(self, initial_state):
        super().__init__(initial_state)
        self.hueristic = 0
        self.path_cost = 0

    def __lt__(self, other) -> bool:
        return self.hueristic < other.hueristic
    
    def __eq__(self, other) -> bool:
        return self.hueristic == other.hueristic and self.current_state == other.current_state

class GeneticAlgorithmState(State):
    def __init__(self, initial_state):
        super().__init__(initial_state)
    
    def __eq__(self, other) -> bool:
        return self.current_state == other.current_state

In [3]:
class Search:
    def __init__(self, initial_state: State):
        self.state = initial_state
        self.explored = set()

    def solve(self):
        raise NotImplementedError("Solve method must be implemented in subclasses")


class UniformCostSearch(Search):
    def __init__(self, initial_state: State):
        super().__init__(initial_state)
        self.frontier = [(0, self.state.current_state)]

    def solve(self):
        while self.frontier:
            cost, current = heapq.heappop(self.frontier)
            if self.state.goal_test(current):
                return current, self.frontier, self.explored
            self.explored.add(tuple(current))
            for successor in self.state.get_successors(current):
                successor_tuple = tuple(successor)
                if successor_tuple not in self.explored:
                    existing_node = next((n for n in self.frontier if tuple(n[1]) == successor_tuple), None)
                    if existing_node and existing_node[0] > cost + 1:
                        self.frontier.remove(existing_node)
                    self.frontier.append((cost + 1, successor))
            heapq.heapify(self.frontier)
            
            print('Size', len(self.frontier))
        return None


class AStarSearch(Search):
    def __init__(self, initial_state: AStarState):
        super().__init__(initial_state)
        self.frontier = [(self.state.count_conflicts(self.state.current_state), 0, self.state.current_state)]
        heapq.heapify(self.frontier)

    def solve(self):
        while self.frontier:
            heuristic, cost, current = heapq.heappop(self.frontier)
            if heuristic == 0:
                return current, self.frontier, self.explored
            self.explored.add(tuple(current))
            for successor in self.state.get_successors(current):
                successor_tuple = tuple(successor)
                if successor_tuple not in self.explored:
                    existing_node = next((n for n in self.frontier if tuple(n[2]) == successor_tuple), None)
                    if existing_node and existing_node[0] + existing_node[1] > self.state.count_conflicts(successor) + cost + 1:
                        self.frontier.remove(existing_node)
                    self.frontier.append((self.state.count_conflicts(successor), cost + 1, successor))
            heapq.heapify(self.frontier)
                
            print(f'Heuristic: {self.frontier[0][0]}')
        return None

class GeneticAlgorithm(Search):
    def __init__(self, initial_state: GeneticAlgorithmState):
        super().__init__(initial_state)
        self.population = [(self.fitness(self.state.current_state), self.state.current_state)]
        heapq.heapify(self.population)
        self.mutation_rate = 0.8
        self.generation = 0
    
    def initialize_population(self, num_queens):
        num_individuals = random.randint(2, num_queens)
        for _ in range(num_individuals):
            individual = [random.randint(0, num_queens - 1) for _ in range(num_queens)]
            self.population.append((self.fitness(individual), individual))
        heapq.heapify(self.population)
    
    def fitness(self, current_state):
        collisions = self.state.count_conflicts(current_state)
        return collisions

    def random_pick(self):
        new_population = []
        n = random.randint(2, len(self.population))
        for i in range(n):
            new_population.append(self.population[i][1])
        return new_population
    
    def crossover(self, parent1, parent2):
        crossover_point = random.randint(0, len(parent1) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2

    def mutate(self, child):
        if random.random() < self.mutation_rate:
            mutate_point = random.randint(0, len(child) - 1)
            mutation = random.randint(0, len(child) - 1)
            child[mutate_point] = mutation
        return child

    def solve(self):
        num_queens = len(self.state.current_state)
        self.initialize_population(num_queens)
            
        for _, parent in self.population:
            if self.state.goal_test(parent):
                return parent, self.population, self.explored

        while self.population:
            selected_children = self.random_pick()
            for i in range(0, len(selected_children), 2):
                if i + 1 < len(selected_children):
                    selected_children[i], selected_children[i + 1] = self.crossover(selected_children[i], selected_children[i + 1])
                    selected_children[i] = self.mutate(selected_children[i])
                    selected_children[i + 1] = self.mutate(selected_children[i + 1])

            self.population = self.population[:-(len(selected_children))]
            self.population.extend([(self.fitness(child), child) for child in selected_children])
            heapq.heapify(self.population)
            
            for _, child in self.population:
                if self.state.goal_test(child):
                    return child, self.population, self.explored
            self.generation += 1
            print(f'Generation: {self.generation} | Cost: {self.population[0][0]}')
        return None


In [4]:
class Console:
    def __init__(self):
        self.frontier = []
        self.explored = set()
        self.initial_state = []
        self.goal_state = None
        self.algorithm = None
        self.size = 0
        self.choice = 0
        self.times = 0
        self.memory = 0

    def __str__(self) -> str:
        print('\nInitial board: ')
        self.print_state(self.initial_state)
     
        if self.goal_state is None:
            return "No solution found."
        else:
            print(f"Solution found:")
            self.print_state(self.goal_state)
            print(f"\nRunning time: {self.times:.4f} miliseconds")
            print(f"Memory usage: {self.memory:.4f} MB")
        
        return ''
    
    def initialize_state(self):
        self.initial_state = [random.randint(0, self.size - 1) for _ in range(self.size)]
    
    def print_state(self, board):
        for i in range(self.size):
            row = ' '.join('Q' if board[i] == j else '-' for j in range(self.size))
            print(row)
        print()

    def get_num_queens(self):
        n = int(input("Enter the number of queens: "))
        self.size = n

    def get_choice(self):
        choice = int(input("Enter the algorithm you want to use (1: UCS, 2: A*, 3: Genetic): "))
        self.choice = choice

    def get_algorithm(self):
        self.initialize_state()

        if self.choice == 1:
            problem = State(self.initial_state)
            self.algorithm = UniformCostSearch(problem)
        elif self.choice == 2:
            problem = AStarState(self.initial_state)
            self.algorithm = AStarSearch(problem)
        elif self.choice == 3:
            problem = GeneticAlgorithmState(self.initial_state)
            self.algorithm = GeneticAlgorithm(problem)

    def execute(self):
        self.get_algorithm()
        self.goal_state, self.frontier, self.explored = self.algorithm.solve()

    def calculate_run_time(self):
        start_time = time.time()  # Start tracking running time
        self.execute()
        self.times = (time.time() - start_time) * 1000

    def calculate_memory(self):
        # Measure memory usage
        frontier = (sys.getsizeof([]) + sum( + sys.getsizeof(node) for node in self.frontier)) / (1024 ** 2)
        explored = (sys.getsizeof(set()) + sum(sys.getsizeof(state) for state in self.explored)) / (1024 ** 2)
        self.memory = frontier + explored
        

    def main(self):
        self.get_num_queens()
        self.get_choice()
        self.calculate_run_time()
        self.calculate_memory()

In [5]:
if __name__ == "__main__":
    console = Console()
    console.main()
    print(console)

Generation: 1 | Cost: 5
Generation: 2 | Cost: 5
Generation: 3 | Cost: 5
Generation: 4 | Cost: 6
Generation: 5 | Cost: 5
Generation: 6 | Cost: 4
Generation: 7 | Cost: 4
Generation: 8 | Cost: 4
Generation: 9 | Cost: 4
Generation: 10 | Cost: 4
Generation: 11 | Cost: 2
Generation: 12 | Cost: 2
Generation: 13 | Cost: 2
Generation: 14 | Cost: 2
Generation: 15 | Cost: 2
Generation: 16 | Cost: 2
Generation: 17 | Cost: 2
Generation: 18 | Cost: 2
Generation: 19 | Cost: 2
Generation: 20 | Cost: 2
Generation: 21 | Cost: 2
Generation: 22 | Cost: 2
Generation: 23 | Cost: 2
Generation: 24 | Cost: 2
Generation: 25 | Cost: 2
Generation: 26 | Cost: 2
Generation: 27 | Cost: 2
Generation: 28 | Cost: 2
Generation: 29 | Cost: 2
Generation: 30 | Cost: 2
Generation: 31 | Cost: 2
Generation: 32 | Cost: 2
Generation: 33 | Cost: 2
Generation: 34 | Cost: 2
Generation: 35 | Cost: 2
Generation: 36 | Cost: 3
Generation: 37 | Cost: 3
Generation: 38 | Cost: 3
Generation: 39 | Cost: 4
Generation: 40 | Cost: 3
Generatio